# Important Notes:

1. The first thing to know about is a "Byte Pair encoding" (BPE). According to huggingface [this does not actually *necessarily* involves bytes](https://huggingface.co/docs/transformers/tokenizer_summary#bytepair-encoding-bpe). Rather, given a corpus $C$ and a base vocabulary $V$ such that $\forall c \in C, \exists (v_1, ..., v_k) \in V^\star s.t. c=v_1\cdots v_k$. The tokenizer will then find the most frequent pair of symbols $\in V$ and merge them, then re-evaluate frequencies with the new $V'$. This repeats until our vocabulary has a desired small enough size.
2. Byte-Level BPE is a special case of the "Byte Pair Encoding" where the symbols are bytes, that is $v_1 = 0b00000000, v_2 = 0b00000001, ..., v_{256} = 0b11111111$. Every unicode character is a sequence of unicode bytes. For example, "🐹" is `0xF0 9F 90 B9`. This means that in theory *any* character can be represented by our model, but the tokens might look really strange to us (see below).  
3. [In the DeBERTa paper](https://arxiv.org/pdf/2006.03654.pdf) they say that they use the BPE vocabulary of [Radford](https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf) and [Liu](https://arxiv.org/pdf/1907.11692.pdf). Liu just references Radford (section 4.4), and in Radford they say:

```
Byte Pair Encoding (BPE) (Sennrich et al., 2015) is a
practical middle ground between character and word level
language modeling which effectively interpolates between
word level inputs for frequent symbol sequences and character level inputs for infrequent symbol sequences. Despite
its name, reference BPE implementations often operate on
Unicode code points and not byte sequences. These implementations would require including the full space of Unicode symbols in order to model all Unicode strings. This
would result in a base vocabulary of over 130,000 before
any multi-symbol tokens are added. This is prohibitively
large compared to the 32,000 to 64,000 token vocabularies
often used with BPE. In contrast, a byte-level version of
BPE only requires a base vocabulary of size 256. However,
directly applying BPE to the byte sequence results in suboptimal merges due to BPE using a greedy frequency based
heuristic for building the token vocabulary. We observed
BPE including many versions of common words like dog
since they occur in many variations such as dog. dog!
dog? . This results in a sub-optimal allocation of limited
vocabulary slots and model capacity. To avoid this, we prevent BPE from merging across character categories for any
byte sequence. We add an exception for spaces which significantly improves the compression efficiency while adding
only minimal fragmentation of words across multiple vocab
token
```
The unicode categories they refer to I believe are [these](https://www.fileformat.info/info/unicode/category/index.htm)

So what is the conclusion here? We don't need to worry about out of vocabulary words, but our tokens might be pretty challenging to interpret.

In [ ]:
from transformers import AutoTokenizer, PreTrainedTokenizer
tokenizer:PreTrainedTokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-base')

In [ ]:
tokenizer.tokenize('🦙')

['ðŁ', '¦', 'Ļ']